# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***
This is a modified version of the previous question and requires you to use an artificial nearest neighbors library

We suggest you to use one of the following:
- [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
- [FAISS](https://github.com/facebookresearch/faiss)
- [Annoy](https://github.com/spotify/annoy.git)

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

You may use any other recommendation system that you wish to use. However,
evaluation script will score your submission by measuring the similarity between
users with the number of common images they appreciated.
The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [2]:
!pip install annoy

     |████████████████████████████████| 646 kB 11.8 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391640 sha256=233cdb83b8b203535486bb46539244603da7d3206ec34509d9d4280777fde83f
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [3]:
!pip install scann

     |████████████████████████████████| 10.6 MB 9.4 MB/s 


In [4]:
!pip install faiss-gpu


     |████████████████████████████████| 85.5 MB 101 kB/s 


In [5]:
import faiss

In [6]:
import annoy
from annoy import AnnoyIndex

In [7]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds, eigs
from sklearn.decomposition import TruncatedSVD
from numpy import dot
from numpy.linalg import norm

In [8]:
import scann

In [9]:
config = pd.read_csv('config.csv').iloc[0]

In [10]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

In [11]:
user = config['id']
k_value = config['k']

### Read the Data

In [12]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [45]:
user_likes = dict()
images=set()
users=set()

In [46]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = line.split()[0]
    item_id = line.split()[1]

    if user_id not in user_likes:
        user_likes[user_id] = list()
    images.add(item_id)
    users.add(user_id)
    user_likes[user_id].append(item_id)

## Data Preprocessing

In [47]:
users=list(users)
images=list(images)

In [48]:
print(len(users))
print(len(images))

63497
178788


In [49]:
user_indices=dict()
image_indices=dict()
for i in range(len(users)):
  user_indices[users[i]]=i

for i in range(len(images)):
  image_indices[images[i]]=i

In [51]:
rows=[]
cols=[]
values=[]
for line in appreciate_data:
    line = line.strip()
    
    user_id = line.split()[0]
    item_id = line.split()[1]
    rows.append(user_indices[user_id])
    cols.append(image_indices[item_id])
values=[1]*len(rows)


In [52]:
rows=np.array(rows)
cols=np.array(cols)
values=np.array(values)

## Matrix Decomposition

In [53]:
cs_matrix=csr_matrix((values, (rows, cols))).tocsr()
cs_matrix.shape

(63497, 178788)

In [22]:
n_comp=1000
while True:
  global m
  svd = TruncatedSVD(n_components=n_comp, random_state=42)
  m=svd.fit(cs_matrix)
  print(svd.explained_variance_ratio_.sum())
  if(svd.explained_variance_ratio_.sum()>=0.8):
    break
  n_comp+=250

In [54]:
svd = TruncatedSVD(n_components=n_comp, random_state=42)
m=svd.fit_transform(cs_matrix)
print(svd.explained_variance_ratio_.sum())


0.45269291373761317


## Calculating Neighbours using Vanilla KNN

In [24]:
def Cosine_similarity(a,b):
  return abs(dot(a,b)/(norm(a)*norm(b)))
def Cosine_distance(a,b):
  return 1-Cosine_similarity(a,b)

In [55]:
def cosine_neighbours(user,k_value=5):
  user_index=user_indices[user]
  user_vec=m[user_index]
  distances=[]
  for i in range(len(m)):
    if i!=user_index:
      distances.append([i,Cosine_distance(user_vec,m[i])])
  distances.sort(key=lambda x:x[1])
  distances=distances[:k_value]
  k_neighbours=[]
  for i in range(k_value):
    index=distances[i][0]
    k_neighbours.append(users[index])
    # print(distances[i][1])
  return k_neighbours



In [57]:
cosine_neighbours(user,k_value)

['1494939', '1973004', '1480429', '2452817', '3660527']

### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

## Calculating Neighbours using Annoy

In [27]:
def neighbours_annoy(userid,k=5):
  f = 1000  #  No of components
  t = AnnoyIndex(f, 'dot')  # Length of item vector that will be indexed
  for i in range(len(m)):
      t.add_item(i, m[i])

  t.build(10) # 10 trees
  user_index=user_indices[userid]

  neighbours_indices=t.get_nns_by_item(user_index, k) # will find the 1000 nearest neighbors
  k_neighbours=[]
  for i in range(len(neighbours_indices)):
    k_neighbours.append(users[neighbours_indices[i]])
  return k_neighbours

In [28]:
neighbours_annoy(user,k_value+1)

[276633, 2679241, 1968571, 2903947, 498932, 2585365]

In [37]:
result_images_annoy=[]
for n_user in neighbours_annoy(user,k_value+1):
    for item_id in user_likes[n_user]:
      if(item_id not in user_likes[user] and item_id not in result_images_annoy):
        result_images_annoy.append([n_user ,item_id])

In [30]:
result_images_annoy

[[2679241, 1522749],
 [2679241, 1588323],
 [2679241, 593596],
 [2679241, 593641],
 [2679241, 573239],
 [2679241, 1588099],
 [2679241, 918715],
 [2679241, 1590293],
 [2679241, 1365843],
 [2679241, 880748],
 [2679241, 1317399],
 [2679241, 381541],
 [2679241, 381580],
 [2679241, 328209],
 [2679241, 923549],
 [2679241, 1461859],
 [2679241, 1351569],
 [2679241, 1333845],
 [2679241, 1122643],
 [2679241, 675905],
 [2679241, 622798],
 [2679241, 631374],
 [2679241, 533391],
 [2679241, 519998],
 [2679241, 539547],
 [2679241, 1591989],
 [2679241, 1579101],
 [2679241, 1123081],
 [2679241, 1501895],
 [2679241, 1612323],
 [2679241, 1463723],
 [2679241, 1608001],
 [2679241, 1609205],
 [2679241, 834640],
 [2679241, 721278],
 [2679241, 506306],
 [2679241, 590990],
 [2679241, 525454],
 [2679241, 1637207],
 [2679241, 1636329],
 [2679241, 1600065],
 [2679241, 1631385],
 [2679241, 1631739],
 [2679241, 1631223],
 [2679241, 1630709],
 [2679241, 1634163],
 [2679241, 1632363],
 [2679241, 1631789],
 [2679241, 1

## Calculating Neighbours using Scann

In [59]:
def neighbours_scan(userid,k=5):
  # normalized_dataset = m / np.linalg.norm(m, axis=1)[:, np.newaxis]
  searcher = scann.scann_ops_pybind.builder(m, 10, "dot_product").score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

  user_index=user_indices[userid]
  neighbours_indices, distances = searcher.search(m[user_index], final_num_neighbors=k)

  k_neighbours=[]
  for i in range(len(neighbours_indices)):
    k_neighbours.append(users[neighbours_indices[i]])
  return k_neighbours


In [60]:
neighbours_scan(user,k_value+1)

['276633', '3628520', '2492899', '666917', '3313136', '2585131']

In [61]:
result_images_scan=[]
for n_user in neighbours_scan(user,k_value+1):
    for item_id in user_likes[n_user]:
      if(item_id not in user_likes[user] and item_id not in result_images_scan):
        result_images_scan.append([n_user ,item_id])

In [39]:
result_images_scan

[[3628520, 1560589],
 [3628520, 1543969],
 [3628520, 1521547],
 [3628520, 1144475],
 [3628520, 1580219],
 [3628520, 1560975],
 [3628520, 1528461],
 [3628520, 1569051],
 [3628520, 1518771],
 [3628520, 1563689],
 [3628520, 1000247],
 [3628520, 1546145],
 [3628520, 1522749],
 [3628520, 945221],
 [3628520, 1571773],
 [3628520, 1568779],
 [3628520, 1553557],
 [3628520, 1576563],
 [3628520, 887427],
 [3628520, 1588323],
 [3628520, 1567311],
 [3628520, 1587481],
 [3628520, 1590493],
 [3628520, 880748],
 [3628520, 1211427],
 [3628520, 1599355],
 [3628520, 1569905],
 [3628520, 1590987],
 [3628520, 1578335],
 [3628520, 1526645],
 [3628520, 1351049],
 [3628520, 1390183],
 [3628520, 1590381],
 [3628520, 1591503],
 [3628520, 1580185],
 [3628520, 923549],
 [3628520, 1590293],
 [3628520, 1569797],
 [3628520, 1596247],
 [3628520, 1591989],
 [3628520, 1579101],
 [3628520, 1593599],
 [3628520, 348151],
 [3628520, 1317399],
 [3628520, 1461859],
 [3628520, 1123081],
 [3628520, 1585141],
 [3628520, 1501895

## Calculating Neighbours using Faiss

In [35]:
def neighbours_faiss(userid,k_v=5):
    index = faiss.IndexFlatL2(len(m[0]))
    m_np=m.astype('float32')
    index.add(m_np)

    user_index=user_indices[userid]

    D,I=index.search(m_np,int(k_v))

    neighbours_indices=I[user_index]
    k_neighbours=[]
    for i in range(len(neighbours_indices)):
      k_neighbours.append(users[neighbours_indices[i]])
    return k_neighbours

In [36]:
neighbours_faiss(user,k_value+1)

[276633, 2452817, 1480429, 1525134, 1459742, 148706]

In [62]:
result_images_faiss=[]
for n_user in neighbours_faiss(user,k_value+1):
    for item_id in user_likes[n_user]:
      if(item_id not in user_likes[user] and item_id not in result_images_faiss):
        result_images_faiss.append([n_user ,item_id])

In [63]:
result_images_faiss

[['2452817', '00455813'],
 ['2452817', '00697415'],
 ['2452817', '00387184'],
 ['2452817', '00337290'],
 ['2452817', '00336527'],
 ['2452817', '00336853'],
 ['2452817', '00336875'],
 ['1480429', '00455813'],
 ['1480429', '00697415'],
 ['1480429', '00337290'],
 ['1480429', '00336853'],
 ['1480429', '00336875'],
 ['1525134', '00455813'],
 ['1525134', '00337290'],
 ['1525134', '00697415'],
 ['1525134', '02330982'],
 ['1525134', '02342028'],
 ['148706', '00455813'],
 ['148706', '00697415'],
 ['148706', '00387184'],
 ['148706', '00337290'],
 ['148706', '00417549'],
 ['148706', '00197269'],
 ['148706', '02272534'],
 ['148706', '02383490'],
 ['148706', '01473969'],
 ['148706', '02096726'],
 ['148706', '02423170'],
 ['148706', '02253840']]

### Answer the following questions:

#### Q1. **Explain how your choice of library works**


Scann performs the vector search in 3 steps. They are:

**1.Partitioning:** During the training phase, Scann divides the dataset into partitions and while testing it returns the top partitions to the next phase.

**2.Scoring:** It computes the distances from the target point to all the data points in the partition.

**3.Rescoring:** It takes the K best distances from scoring and computes them more accurately. From these distances, the top K are chosen

#### Q2. **Compare your choice of library with vanilla KNN.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***


For every query point, the traditional nearest neighbours calculates the distance with all the potential target objects and finds the closest ones. Most of the modern implementations use Kd tree and Ball trees. A kd tree partitions the vector space into tree and reduces the complexity of searching a vector to O(log n). A Ball tree also uses tree based data structures nut rather it partitions the data into hyperspheres. It can search for data with complexity of O(n logn).
While calculating distances, the algorithm has to traverse all the dimensions. Let d be the number of dimensions. The overall time complexity during training phase is O(k(d + logn)) where k is the number os testing points.

#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

**SaNN**

As the dataset given has less than 100k data points, partitioning here is not required.
Scoring is performed using Asymmetric Hashing. AH gives better speed/accuracy tradeoffs.
At rescoring phase, increasing reordering_num_neighbors increases the accuracy at the cost of speed( given that the reordering_num_neighbors should be greater than k.

**Faiss**

Faiss is used for similarity search and clustering. It assumes that vectors are identified by integers and vectors can be compared with L2(Euclidean distance) or dot products.
It is built around index type that stores a set of vectors and provides a function to search in them.

**Annoy**

Annoy(Approximate Nearest Neighbour Search) is used to search for points in space that are close to query point. It is useful to find the nearest neighbours when you have multiple CPU's making the job parallel.

TradeOff- The tuning parameters in Annoy are n_trees and Search_k. The n_trees will affect the build time, memory and search_k will affect the running time. 



### Open the output file to write all the lines to the file

In [64]:
outFile = open(config['output_file'], 'w')

for user_id in neighbours_scan(user,k_value+1):
    # user_id = list(user_likes.keys())[n_user]
    for item_id in user_likes[user_id]:
      if(item_id not in user_likes[user]):
        outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()